In [2]:
import pandas as pd

# 시리즈 생성. 숫자 인덱스가 자동으로 생성 
s2 = pd.Series(['조조', 96, 72, 91, 94, 96], index=['이름', '통솔', '무력', '지력', '정치', '매력'])
s2

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 훈련용 데이터 불러오기 
train = pd.read_csv("https://raw.githubusercontent.com/developer-sdk/kaggle-python-beginner/master/datas/kaggle-titanic/train.csv")
train.head(5)
# Embarked는 승선한 곳을 말하며, C(Cherbourg), Q(Queenstown), S(Southampton)를 뜻함.
# 인덱스 정보: https://thebook.io/006723/ch11/01/

In [ ]:
train.isnull().sum()

In [ ]:
def show_pie_chart(df, col_name):
    colname_survived = survived_crosstab(train, col_name)
    pie_chart(colname_survived)
    return colname_survived

def survived_crosstab(df, col_name):
    '''col_name과 Survived간의 교차도표 생성'''
    feature_survived = pd.crosstab(df[col_name], df['Survived'])
    feature_survived.columns = feature_survived.columns.map({0:"Dead", 1:"Alive"})
    return feature_survived

def pie_chart(feature_survived):
    '''
    pie_chart 생성
    pcol, prow = 차트를 출력할 개수. pcol * prow 만큼의 차트 출력 
    '''
    frows, fcols = feature_survived.shape
    pcol = 3
    prow = (frows/pcol + frows%pcol)
    plot_height = prow * 2.5
    plt.figure(figsize=(8, plot_height))

    for row in range(0, frows):
        plt.subplot(prow, pcol, row+1)

        index_name = feature_survived.index[row]
        plt.pie(feature_survived.loc[index_name], labels=feature_survived.loc[index_name].index, autopct='%1.1f%%')
        plt.title("{0}' survived".format(index_name))

    plt.show()